# Prepare the dataset

# Run token analysis and topic analysis

In [9]:
import re
import csv
import warnings
import math
import numpy
import numpy as np
import collections
import string
import scipy
from scipy import sparse
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
import sys
import csv


# preprocessing method
def string_tokenise(string):  # return list
    result = re.findall(r"\w+", string)
    return result


def case_fold(list1):  # return list
    result = [word.lower() for word in list1]
    #     string = ' '.join([str(elem) for elem in list1])
    #     result = string.lower().split() #lower() is the same as casefold()
    return result


def stopping(list1):  # return list
    stopfile = open("englishST.txt", 'r')
    stopwords = stopfile.read().split()
    result = [items for items in list1 if items not in stopwords]
    return result


def normalise(list1):  # return list
    result = []
    for item in list1:
        result.append(stem(item))
    return result


def pre_process(string):  # return list
    result = normalise(stopping(case_fold(string_tokenise(string))))
    return result


# find out the terms that appear >= 10 times
# input: the name of the corpus | return: a list including the freq term
def find_freq_terms10(corpus_name):
    # put all tokens into a single list
    all_tokens = []
    for name, content in verses_list:  # e.g. verses_list[0] == ['Quran', ['prais', 'allah', 'lord', 'world']]
        if name == corpus_name:
            all_tokens += content

    count = Counter(all_tokens)  # count the how many times the terms appears, Counter(['prais'])['prais'] == 1
    freq_terms_list = []
    freq_terms_list.append(corpus_name)  # put corpus name at the first one in the list
    for term in count:
        if count[term] >= 10:  # if the term appering more than 10 times
            freq_terms_list.append(term)
    return freq_terms_list


# calculate MI
def MI(N11, N01, N10, N00):
    N = N11 + N01 + N10 + N00
    N1_ = N11 + N10  # N1.
    N_1 = N11 + N01  # N.1
    N0_ = N00 + N01  # N0.
    N_0 = N00 + N10  # N.0
    # To avoid zero division
    output = 0
    if N11 != 0:
        output += (N11 / N) * np.log2((N * N11) / (N1_ * N_1))
    if N01 != 0:
        output += (N01 / N) * np.log2((N * N01) / (N0_ * N_1))
    if N10 != 0:
        output += (N10 / N) * np.log2((N * N10) / (N1_ * N_0))
    if N01 != 0:
        output += (N00 / N) * np.log2((N * N00) / (N0_ * N_0))
    return output


# Calculate Chi_squared
def Chi_squared(N11, N01, N10, N00):
    N = N11 + N01 + N10 + N00
    return (N * ((N11 * N00 - N10 * N01) ** 2)) / ((N11 + N01) * (N11 + N10) * (N10 + N00) * (N01 + N00))


verses_list = []
# a list to store the preprocessed corpora in list structure, e.g.['Quran', ['prais', 'allah', 'lord', 'world']]

Poynter = open('PoynterCovid19Database_Reference_Article.csv', 'r')
englishST = open('englishST.txt', 'r')  # read stop words
corpora = csv.reader(Poynter, delimiter=',')
stopword_list = englishST.read().split()  # read stop words into a list

csv.field_size_limit(sys.maxsize)
for (docID,content,accuracy,date,region,explanation,
     reference_url,reference_html,reference_text) in corpora:
    p_content = pre_process(explanation)
    verses_list.append(['Poynter', p_content])
    # strore preprocessed corpora in this list structure, e.g.['Quran', ['prais', 'allah', 'lord', 'world']]

# find terms that appears >= 10 times and put them in to one list
Quran_terms = find_freq_terms10(corpus_name='Quran')  # NB the first element is the name of corpus
OT_terms = find_freq_terms10(corpus_name='OT')  # NB the first element is the name of corpus
NT_terms = find_freq_terms10(corpus_name='NT')  # NB the first element is the name of corpus
Poynter_term = find_freq_terms10(corpus_name='Poynter')
corpora_terms = [Poynter_term]
# put in to one list [['Quran',<Quran_terms>],[<'OT',OT_terms>],['NT',<NT_terms>]]

# identify top 10 highest scoring words for each method for each corpus
# for corpus_terms in corpora_terms:
#     # for the frequently appearing terms of each corpus. corpus_terms is a list likes [<Quran_terms>]
#     corpus_of_terms = corpus_terms[0]  # recall the first element is the name of the corpus
#     MI_dict = {}  # a dictionary with structure {term:MI}
#     Chi_dict = {}  # a dictionary with structure {term:Chi Squared}
#     for term in corpus_terms[1:]:
#         N11 = 0
#         N01 = 0
#         N10 = 0
#         N00 = 0
#         for corpus_of_docs, p_content in verses_list:
#             if corpus_of_terms == corpus_of_docs:
#                 # number of documents that contain the word are in the class (N11)
#                 if term in p_content:
#                     N11 = N11 + 1
#                 # number of documents that don't contain the word are in the class(N01)
#                 else:
#                     N01 = N01 + 1
#             else:
#                 # number of documents where this term present in other corpora(N10)
#                 if term in p_content:
#                     N10 = N10 + 1
#                 # number of documents where this term does not present in other corpora(N00)
#                 else:
#                     N00 = N00 + 1
#         MI_dict[term] = MI(N11, N01, N10, N00)
#         Chi_dict[term] = Chi_squared(N11, N01, N10, N00)

#     # sort the dictionary
#     sorted_MI_dict = sorted(MI_dict.items(), key=lambda x: x[1], reverse=True)
#     sorted_Chi_dict = sorted(Chi_dict.items(), key=lambda x: x[1], reverse=True)
#     print('The top 10 highest Mutual Information score of ' + corpus_of_terms)
#     print(sorted_MI_dict[:10])
#     print('\n')
#     print('The top 10 highest Chi_squared score of ' + corpus_of_terms)
#     print(sorted_Chi_dict[:10])
#     print('\n')



In [11]:
# Implement IDA, identify the topic that has the highest average score for the three corpora (3 topics).
# For each of those three topics, find the top 10 tokens with highest probability of belonging to that topic.
common_texts = [item[1] for item in verses_list]
# Create a corpus from a list of texts
common_dictionary = Dictionary(common_texts)
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=20, random_state=1000, id2word=common_dictionary)
# get the topic scores for each document
docs_topic_score_list = lda.get_document_topics(bow=common_corpus, minimum_probability=0.00)

# identify top3 topics
for corpus_name in ['Poynter']:
    # compute the topic probabilities
    topic_prob = []  # topic_prob contains the topic prob for each document in one corpus
    for verse, prob in zip(verses_list, docs_topic_score_list):
        if verse[0] == corpus_name:  # eg of verse: ['Quran', ['prais', 'allah', 'lord', 'world']], where verses[
            # 0]=='Quran'
            topic_prob.append(prob)  # add the topic prob for one doc into the list
    print('docs_topic_score_list of ' + corpus_name)
    # compute overall topic probabilities for each corpus by averaging the
    # topic probabilities for all documents belonging to each corpus
    overall_topic_prob = np.mean(topic_prob, axis=0)
    top3_topics = sorted(overall_topic_prob, reverse=True, key=lambda x: x[1])[
                  :10]  # sort the overall_topic_prob to identify top3
    for topic in top3_topics:  # eg of topic: array([19.        ,  0.08826874])
        print(topic)
        print(lda.print_topic(
            int(topic[0])))  # print the the top 10 tokens and their probability scores for each of the 3 topics
    print('\n')

docs_topic_score_list of Poynter
[18.          0.07362767]
0.061*"coronavirus" + 0.050*"covid" + 0.026*"vaccin" + 0.025*"virus" + 0.017*"flu" + 0.017*"infect" + 0.014*"evid" + 0.013*"cure" + 0.010*"diseas" + 0.009*"studi"
[13.          0.07190815]
0.050*"covid" + 0.040*"evid" + 0.024*"coronavirus" + 0.022*"cure" + 0.019*"drug" + 0.018*"prevent" + 0.017*"diseas" + 0.017*"scientif" + 0.016*"health" + 0.015*"virus"
[5.        0.0703799]
0.030*"polic" + 0.026*"fals" + 0.021*"messag" + 0.015*"govern" + 0.013*"video" + 0.012*"inform" + 0.011*"peopl" + 0.010*"covid" + 0.009*"issu" + 0.009*"citi"
[2.         0.06900531]
0.044*"video" + 0.041*"photo" + 0.039*"show" + 0.025*"coronavirus" + 0.017*"covid" + 0.012*"imag" + 0.011*"2019" + 0.010*"2018" + 0.010*"anim" + 0.010*"vaccin"
[6.         0.05857888]
0.059*"20" + 0.038*"coronavirus" + 0.030*"3" + 0.028*"video" + 0.022*"2019" + 0.019*"imag" + 0.018*"2" + 0.010*"current" + 0.010*"covid" + 0.010*"ncov"
[17.          0.05780697]
0.064*"video" + 0.